In [1]:
import torch
import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt
import torchtext

In [2]:
##Change working directory to talk-berty-to-me root
import os
os.chdir("D:/University/Projects/AML/talk-berty-to-me")

In [3]:
data_gutenberg = pd.read_csv('data/books_and_genres.csv')

In [4]:
dev_data = data_gutenberg.sample(frac=0.1, random_state=42)

In [7]:
# universal_set = set()
# def parse_set(string_set):
#     return ast.literal_eval(string_set)

# for string in list(zip(dev_data['genres'])):
#     parsed_set = parse_set(string[0])
#     universal_set = universal_set.union(parsed_set)
# universal_set_list = list(universal_set)

In [8]:
# selected_genres = ['fiction', 'classics', 'historical', '20th-century', 'non-fiction', 'literature', 'history']

In [9]:
# def OneHotEncodeGenres(genres):
#     return [1 if genre in genres else 0 for genre in selected_genres]


# dev_data.loc[:,'genre_one_hot'] = dev_data['genres'].apply(lambda x: OneHotEncodeGenres(x))

In [32]:
# dev_data.drop(columns=['genres'], inplace=True)

In [5]:
dev_data.head()

,Unnamed: 0,title,text,genres
3686,3686,die ungleichen schalen,"Produced by Markus Brenner, Marina Lukas and t...",{'plays'}
3526,3526,galatea,"Produced by Carlo Traverso, Claudio Paganelli ...","{'contemporary', 'literary-fiction', 'romance'..."
5723,5723,surprising stories,"Produced by Susan Skinner, Marilynda Fraser-Cu...","{'unfinished', 'short-stories'}"
8603,8603,hypatia,Produced by P. J. Riddick\n\n\n\n\n\n\n\n\n\nH...,"{'literary-fiction', 'christian', 'history', '..."
3501,3501,the babes in the wood,"Produced by Jonathan Niehof, Suzanne Shell and...","{'picture-books', 'classics', 'fiction', '20th..."


In [6]:
#Building vocabulary
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')
vocab_iter = iter(dev_data.loc[:,'text'] + dev_data.loc[:,'title'] + dev_data.loc[:,'genres'])
def yield_tokens(train_iter):
    for text in train_iter:
        if not isinstance(text, str):
            if type(text) == list:
                for t in text:
                    yield tokenizer(t)
            continue
        yield tokenizer(text)

In [7]:
vocab = build_vocab_from_iterator(
    yield_tokens(vocab_iter), specials=["<unk>"], min_freq=1000)
vocab.set_default_index(vocab["<unk>"])

In [8]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\setul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\setul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
from rake_nltk import Rake
r = Rake()
def rake_extract(text):
    r.extract_keywords_from_text(str(text))
    ranked_phrases_with_scores = r.get_ranked_phrases_with_scores()
    sorted_phrases = sorted(ranked_phrases_with_scores, key=lambda x: x[0], reverse=True)
    if len(sorted_phrases) == 0:
        return ""
    return sorted_phrases[0][1]

Possible improvement is to remove proper nouns before employing rake. Will take more effort.
Reduce dataset to just English books. Improve text cleaning to remove redundant symbols.

In [10]:
dev_data.loc[:,'sentences'] = dev_data.loc[:,'text'].apply(lambda x: nltk.tokenize.sent_tokenize(str(x)))
dev_data = dev_data.explode('sentences')
dev_data.loc[:,'keywords'] = dev_data.loc[:,'sentences'].apply(lambda x: rake_extract(x))
dev_data = dev_data.loc[:,['title', 'sentences', 'keywords', 'genres']]
dev_data.reset_index(drop=True, inplace=True)

In [11]:
dev_data['label_sentences'] = dev_data.groupby('title')['sentences'].shift(-1)

dev_data = dev_data.dropna(subset=['label_sentences'])

In [12]:
dev_data['label_keywords'] = dev_data.groupby('title')['keywords'].shift(-1)

dev_data = dev_data.dropna(subset=['label_keywords'])

In [20]:
dev_data.iloc[14800:14806]

,title,sentences,keywords,genres,label_sentences,label_keywords
14806,hypatia,You drove from her\nrecollection the faith in ...,baptized !',"{'literary-fiction', 'christian', 'history', '...","'So much the better for her, if the recollecti...",recollection
14807,hypatia,"'So much the better for her, if the recollecti...",recollection,"{'literary-fiction', 'christian', 'history', '...",Better to wake unexpectedly in Gehenna\nwhen y...,wake unexpectedly
14808,hypatia,Better to wake unexpectedly in Gehenna\nwhen y...,wake unexpectedly,"{'literary-fiction', 'christian', 'history', '...","And\nas for leaving her untaught, on your own ...",much already
14809,hypatia,"And\nas for leaving her untaught, on your own ...",much already,"{'literary-fiction', 'christian', 'history', '...",Wiser it would be in you to curse your parents...,ten years
14810,hypatia,Wiser it would be in you to curse your parents...,ten years,"{'literary-fiction', 'christian', 'history', '...","Come now, don't be angry with me.",come
14811,hypatia,"Come now, don't be angry with me.",come,"{'literary-fiction', 'christian', 'history', '...","The old\nJewess is your friend, revile her as ...",old jewess


In [80]:
###Functional but conceptually iffy
# from torch.nn.utils.rnn import pad_sequence
# def collate_sentences(batch):
#     titles, genres, sentences, label_sentences = zip(*batch)
#     input_sentence = [tokenizer(t) + ['<BOS>'] + tokenizer(s) + ['<EOS>'] +
#                        tokenizer(g) for t, s, g in zip(titles, sentences, genres)]
#     label_sentence = [tokenizer(s) for s in label_sentences]
#     input_tensor = pad_sequence([torch.tensor(vocab.lookup_indices(t)) for t in input_sentence],
#                                  padding_value=vocab['<pad>'], batch_first=True)
#     label_tensor = pad_sequence([torch.tensor(vocab.lookup_indices(t)) for t in label_sentence],
#                                     padding_value=vocab['<pad>'], batch_first=True)
#     return input_tensor, label_tensor

In [85]:
# def collate_keywords(batch):
#     titles, genres, keywords, label_keywords = zip(*batch)
#     input_keywords = [tokenizer(t) + ['<BOS>'] + tokenizer(k) + ['<EOS>'] +
#                        tokenizer(g) for t, k, g in zip(titles, keywords, genres)]
#     label_keywords = [tokenizer(k) for k in label_keywords]
#     input_tensor = pad_sequence([torch.tensor(vocab.lookup_indices(k)) for k in input_keywords],
#                                  padding_value=vocab['<pad>'], batch_first=True)
#     label_tensor = pad_sequence([torch.tensor(vocab.lookup_indices(k)) for k in label_keywords],
#                                     padding_value=vocab['<pad>'], batch_first=True)
#     return input_tensor, label_tensor

In [25]:
# from torch.nn.utils.rnn import pad_sequence
# def collate_context(batch):
#     titles, genres = zip(*batch)
#     context = [tokenizer(g) + ['<BOS>'] + tokenizer(t) + ['<EOS>'] 
#                         for t, g in zip(titles, genres)]
#     encoder_tensor = pad_sequence([torch.tensor(vocab.lookup_indices(t)) for t in context],
#                                  padding_value=vocab['<pad>'], batch_first=True)
#     return encoder_tensor

In [29]:
# context_dataloader = torch.utils.data.DataLoader(dev_data.loc[:,['title', 'genres']].values, batch_size=8, shuffle=True, collate_fn=collate_context)

In [32]:
# from torch.nn.utils.rnn import pad_sequence
# def collate_sentences(batch):
#     sentences, label_sentences = zip(*batch)
#     input_sentence = [['<BOS>'] + tokenizer(s) + ['<EOS>'] for s in sentences]
#     label_sentence = [['<BOS>'] + tokenizer(s) + ['<EOS>'] for s in label_sentences]
#     input_tensor = pad_sequence([torch.tensor(vocab.lookup_indices(t)) for t in input_sentence],
#                                  padding_value=vocab['<pad>'], batch_first=True)
#     label_tensor = pad_sequence([torch.tensor(vocab.lookup_indices(t)) for t in label_sentence],
#                                     padding_value=vocab['<pad>'], batch_first=True)
#     return input_tensor, label_tensor

In [91]:
def collate_batch(batch):
    titles, genres, sentences, label_sentences = zip(*batch)
    context = [tokenizer(g) + ['<BOS>'] + tokenizer(t) + ['<EOS>'] 
                        for t, g in zip(titles, genres)]
    input_sentence = [['<BOS>'] + tokenizer(s) + ['<EOS>'] for s in sentences]
    label_sentence = [['<BOS>'] + tokenizer(s) + ['<EOS>'] for s in label_sentences]
    input_tensor = pad_sequence([torch.tensor(vocab.lookup_indices(t)) for t in input_sentence],
                                 padding_value=vocab['<pad>'], batch_first=True)
    label_tensor = pad_sequence([torch.tensor(vocab.lookup_indices(t)) for t in label_sentence],
                                    padding_value=vocab['<pad>'], batch_first=True)
    encoder_tensor = pad_sequence([torch.tensor(vocab.lookup_indices(t)) for t in context],
                                 padding_value=vocab['<pad>'], batch_first=True)
    return input_tensor, label_tensor, encoder_tensor
    

In [177]:
def collate_batch_v2(batch):
    titles, genres, sentences, label_sentences = zip(*batch)
    context = [tokenizer(g) + ['<BOS>'] + tokenizer(t) + ['<EOS>'] +
               ['<BOS>'] + tokenizer(s) + ['<EOS>'] for t, g,
                s in zip(titles, genres, sentences)]
    # input_sentence = [['<BOS>'] + tokenizer(s) + ['<EOS>'] for s in sentences]
    label_sentence = [['<BOS>'] + tokenizer(s) + ['<EOS>'] for s in label_sentences]
    # input_tensor = pad_sequence([torch.tensor(vocab.lookup_indices(t)) for t in input_sentence],
    #                              padding_value=vocab['<pad>'], batch_first=True)
    label_tensor = pad_sequence([torch.tensor(vocab.lookup_indices(t)) for t in label_sentence],
                                    padding_value=vocab['<pad>'], batch_first=True)
    encoder_tensor = pad_sequence([torch.tensor(vocab.lookup_indices(t)) for t in context],
                                 padding_value=vocab['<pad>'], batch_first=True)
    return label_tensor, encoder_tensor

In [22]:
#From HW4
from torch.utils.data import Sampler
class BatchSequentialSampler(Sampler):
    r"""Samples batches, s.t. the ith elements of each batch are sequential.

    Args:
        data_source (Dataset): dataset to sample from
    """

    def __init__(self, data_source, batch_size):
        self.data_source = data_source
        self.batch_size = batch_size
        
    def __iter__(self):
        num_batches = len(self.data_source)//self.batch_size
        for i in range(num_batches):
            for j in range(self.batch_size):
                yield(j * num_batches + i)

    def __len__(self):
        return (len(self.data_source)//self.batch_size) * self.batch_size

In [33]:
# sent_sampler = BatchSequentialSampler(dev_data.loc[
#     :,['sentences', 'label_sentences']], 8)

In [92]:
batch_sampler = BatchSequentialSampler(dev_data.loc[:,['title', 'genres', 'sentences', 'label_sentences']], 8)

In [178]:
batch_sampler_v2 = BatchSequentialSampler(dev_data.loc[:,['title', 'genres', 'sentences', 'label_sentences']], 8)

In [94]:
batch_dataloader = torch.utils.data.DataLoader(dev_data.loc[:,['title', 'genres', 'sentences', 'label_sentences']].values, batch_size=8, collate_fn=collate_batch, sampler=batch_sampler)

In [179]:
batch_dataloader_v2 = torch.utils.data.DataLoader(dev_data.loc[:,['title', 'genres', 'sentences', 'label_sentences']].values, batch_size=8, collate_fn=collate_batch_v2, sampler=batch_sampler_v2)

In [180]:
for idx, (label_tensor, context_tensor) in enumerate(batch_dataloader_v2):
    #print(input_tensor.shape)
    print(label_tensor.shape)
    print(context_tensor.shape)
    break

torch.Size([8, 41])
torch.Size([8, 66])


In [34]:
# sent_dataloader = torch.utils.data.DataLoader(dev_data.loc[
#     :,['sentences', 'label_sentences']].values, batch_size=8,
#       shuffle=False, collate_fn=collate_sentences, sampler=sent_sampler)

In [86]:
# keyword_dataloader = torch.utils.data.DataLoader(dev_data.loc[
#     :,['title', 'genres', 'keywords', 'label_keywords']].values, batch_size=8,
#       shuffle=False, collate_fn=collate_keywords)

In [87]:
# for idx, (input_tensor, label_tensor) in enumerate(keyword_dataloader):
#     print(input_tensor.shape)
#     print(label_tensor.shape)
#     break

torch.Size([8, 31])
torch.Size([8, 21])


In [38]:
VECTOR_CACHE_DIR = '/Users/setul/mlpp23/.vector_cache'
glove = torchtext.vocab.GloVe('6B', cache=VECTOR_CACHE_DIR)
glove_vectors = glove.get_vecs_by_tokens(vocab.get_itos())

In [36]:
import torch.nn as nn
import torch.nn.functional as F


In [37]:
IS_CUDA = torch.cuda.is_available()
if IS_CUDA:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [137]:
class BiRNN_encoder(nn.Module):
    def __init__ (self, embedding_dim, hidden_dim,
                  vocab_size, num_layers=2, type_rnn = 'LSTM', bidirectional = True,
                  dropout = 0.3, pad_idx = 0):
        super(BiRNN_encoder, self).__init__()
        self.rnns = []
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        hidden_size = hidden_dim * 2 if bidirectional else hidden_dim
        input_size = embedding_dim
        for _ in range(num_layers):
            if type_rnn == 'LSTM':
                rnn = nn.LSTM(input_size, hidden_size, 1, dropout = dropout,
                               bidirectional = bidirectional, batch_first=True)
            elif type_rnn == 'GRU':
                rnn = nn.GRU(input_size, hidden_size, 1, dropout = dropout,
                              bidirectional = bidirectional, batch_first=True)
            self.rnns.append(rnn)
            input_size = hidden_size*2 if bidirectional else hidden_size
        self.rnns = nn.ModuleList(self.rnns)
        self.dropout = nn.Dropout(dropout)
        self.type_rnn = type_rnn
        self.bidirectional = bidirectional
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

    def forward(self, input, hidden = None):
        embedded = self.embedding(input)
        embedded = self.dropout(embedded)
        rnn_input = embedded
        for idx, rnn in enumerate(self.rnns):
            output, hidden_output = rnn(rnn_input, hidden)
            hidden = hidden_output
            rnn_input = output
        if self.type_rnn == 'LSTM':
            hidden = hidden[0]
        if self.bidirectional:
            hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
        return output, hidden

In [181]:
class BiRNN_decoder(nn.Module):
    def __init__(self, hidden_size, vocab_size, embedding_dim, batch_size =8, num_layers = 1, nonlinearity = 'tanh',
                 dropout = 0.3, bidirectional = False, batch_first = True, pad_idx = 0):
        super(BiRNN_decoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.dropout = nn.Dropout(dropout)
        self.rnn = nn.LSTM(embedding_dim, hidden_size, num_layers, dropout = dropout,
                           bidirectional = bidirectional, batch_first = batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.nonlinearity = nonlinearity
        self.batch_size = batch_size
        self.vocab_size = vocab_size

    def forward(self, input, hidden = None):
        decoder_input = input.unsqueeze(0)
        embedded = self.embedding(decoder_input)
        embedded = self.dropout(embedded)
        output, hidden_output = self.rnn(embedded, hidden)
        prediction = self.linear(output.squeeze(0))
        return prediction, hidden_output[0]

In [193]:
encoder = BiRNN_encoder(300, 600, len(vocab), num_layers=2, type_rnn = 'LSTM', bidirectional = True, dropout = 0.3, pad_idx = vocab['<pad>'])
decoder = BiRNN_decoder(600, len(vocab), 300, batch_size = 8, num_layers = 1, nonlinearity = 'tanh', dropout = 0.3, bidirectional = False, batch_first = True, pad_idx = vocab['<pad>'])

c:\Users\setul\miniconda3\envs\talk-berty\lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


My original code above

In [184]:
import random
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        #Hidden_dim of encoder and decoder should be the same
    
    def forward(self, trg, src, teacher_forcing_ratio = 0.5):
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = decoder.vocab_size
        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)
        encoder_output, hidden = self.encoder(src)
        hidden = hidden.unsqueeze(0)
        hidden = hidden.repeat(1, 1, 1) #first variable is decoder num layers
        input = trg[:,0]
        for t in range(1, trg_len):
            output, hidden = self.decoder(input, hidden)
            outputs[:,t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[:,t] if teacher_force else top1
        return outputs



In [ ]:
# encoder = BiRNN_encoder(300, 600, len(vocab), 2, 'LSTM', True, 0.3, 0).to(device)
# decoder = BiRNN_decoder(300, len(vocab), 300, 1, 'tanh', 0.3, False, False,0).to(device)

In [194]:
model = Seq2Seq(encoder, decoder, device).to(device)

In [195]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): BiRNN_encoder(
    (embedding): Embedding(4854, 300, padding_idx=0)
    (rnns): ModuleList(
      (0): LSTM(300, 1200, batch_first=True, dropout=0.3, bidirectional=True)
      (1): LSTM(2400, 1200, batch_first=True, dropout=0.3, bidirectional=True)
    )
    (dropout): Dropout(p=0.3, inplace=False)
  )
  (decoder): BiRNN_decoder(
    (embedding): Embedding(4854, 300, padding_idx=0)
    (dropout): Dropout(p=0.3, inplace=False)
    (rnn): LSTM(300, 600, batch_first=True, dropout=0.3)
    (linear): Linear(in_features=600, out_features=4854, bias=True)
  )
)

In [196]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 56,992,854 trainable parameters


In [198]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        trg,src = batch
        trg = trg.to(device)
        src = src.to(device)
        
        optimizer.zero_grad()
        # trg = [sen_len, batch_size]
        # output = [trg_len, batch_size, output_dim]
        output = model(src, trg)
        output_dim = output.shape[-1]
        
        # transfrom our output : slice off the first column, and flatten the output into 2 dim.
        output = output[1:].view(-1, output_dim) 
        trg = trg[1:].view(-1)
        # trg = [(trg_len-1) * batch_size]
        # output = [(trg_len-1) * batch_size, output_dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [189]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
        
        for i, batch in enumerate(iterator):
            trg,src = batch
            
            output = model(src, trg, 0) # turn off teacher forcing.
            
            # trg = [sen_len, batch_size]
            # output = [sen_len, batch_size, output_dim]
            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            
            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
            
    return epoch_loss / len(iterator)

In [199]:
def epoch_time(start_time, end_time):
    
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time  / 60)
    elapsed_secs = int(elapsed_time -  (elapsed_mins * 60))
    return  elapsed_mins, elapsed_secs

In [200]:
import time
N_EPOCHS = 10

CLIP = 1
pad_idx = vocab['<pad>']

best_valid_loss = float('inf')
train_iter = batch_dataloader_v2
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)


for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iter, optimizer, criterion, CLIP)
    #valid_loss = evaluate(model, valid_iter, criterion)
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'Seq2SeqModel.pt')
    print(f"Epoch: {epoch+1:02} | Time {epoch_mins}m {epoch_secs}s")
    print(f"\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:.3f} | Valid PPL: {math.exp(valid_loss):7.3f}")

IndexError: index 1 is out of bounds for dimension 0 with size 1

My original code below

In [156]:
import torch
import torch.nn as nn
import torch.optim as optim
import random

GRAD_CLIP = 1.
NUM_EPOCHS = 3
TEACHER_FORCING_RATIO = 0.5
MAX_LENGTH = 12
BATCH_SIZE = 8

loss_fn = nn.CrossEntropyLoss(ignore_index=vocab['<pad>'])  # Ignore padding token for loss calculation
learning_rate = 0.001
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
val_losses = []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

for epoch in range(NUM_EPOCHS):
    encoder.train()
    decoder.train()
    total_loss = 0

    for i, batch in enumerate(batch_dataloader):
        src, trg, context = batch
        src, trg, context = src.to(device), trg.to(device), context.to(device)
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        output, hidden = encoder(context)
        hidden = repackage_hidden(hidden)  # Prepare hidden state for decoding
        decoder_hidden = hidden
        decoder_input = torch.tensor([vocab['<BOS>']] * BATCH_SIZE, device=device)  # Initialize decoder input
        decoder_input = decoder_input.unsqueeze(0)
        decoder_hidden = decoder_hidden.unsqueeze(0)
        loss = 0
        for t in range(1, trg.size(1)):  # Iterate through the target sequence
            output, hidden, _ = decoder(input, hidden)
            loss += loss_fn(output.squeeze(1), trg[:, t])
            teacher_force = random.random() < TEACHER_FORCING_RATIO
            top1 = output.argmax(2).squeeze(1)
            input = trg[:, t] if teacher_force else top1

        loss /= trg.size(1)  # Normalize loss by sequence length
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(encoder.parameters(), GRAD_CLIP)
        torch.nn.utils.clip_grad_norm_(decoder.parameters(), GRAD_CLIP)

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(batch_dataloader)
    print(f'Epoch {epoch+1}, Loss: {average_loss:.4f}')
    val_losses.append(average_loss)

RuntimeError: For unbatched 2-D input, hx and cx should also be 2-D but got (1-D, 1-D) tensors

In [147]:
import torch
import torch.nn as nn
import torch.nn.functional as F

MAX_LENGTH = 10

class BiRNN_decoder(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size,
                  output_dim, dropout = 0.3):
        super(BiRNN_decoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, 1,
                            dropout = dropout, batch_first=True)
        self.linear = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        self.output_dim = output_dim

    def forward(self, input, encoder_hidden, target= None):
        batch_size = input.size(0)
        decoder_hidden = encoder_hidden
        decoder_input = torch.tensor([vocab['<BOS>']] * batch_size,
                                      device=device).view(batch_size, 1)
        print('decoder_input_post', decoder_input.shape)
        decoder_outputs = []
        if target is None:
            count = MAX_LENGTH
        else:
            count = target.size(1)
        for i in range(count):
            decoder_output, decoder_input = self.forward_step(decoder_input,decoder_hidden)
            decoder_outputs.append(decoder_output)
            if target is not None:
                decoder_input = target[:, i].view(batch_size, 1)
            else:
                _, prediction = decoder_output.topk(1)
                decoder_input = prediction.squeeze().detach().view(-1, 1)
            decoder_outputs = torch.cat(decoder_outputs, dim=1)
            decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden, None

    def forward_step(self, decoder_input, decoder_hidden):
        embedded = self.embedding(decoder_input)
        embedded = self.dropout(embedded)
        ###Experiment with this
        # decoder_hidden = decoder_hidden.unsqueeze(1)
        decoder_output, decoder_hidden = self.rnn(embedded, decoder_hidden)
        decoder_output = self.linear(decoder_output)
        return decoder_output, decoder_input





        #     print('decoder_layer', i)
        #     embedded = self.embedding(decoder_input)
        #     embedded = self.dropout(embedded)
        #     print("embedded", embedded.shape)
        #     print("decoder_hidden", decoder_hidden.shape)
        #     decoder_output, decoder_hidden = self.rnn(embedded, decoder_hidden)
        #     decoder_outputs.append(decoder_output)
        #     _, prediction = decoder_output.topk(1)
        #     if target is not None:
        #         decoder_input = target[:, i].view(batch_size, 1)
        #     else:
        #         decoder_input = prediction.squeeze().detach().view(-1, 1)
        # decoder_outputs = F.log_softmax(torch.cat(decoder_outputs, dim=1), dim=2)
        # return decoder_outputs, decoder_hidden, None

### First create seq2seq model unconstrained by keywords

In [148]:
encoder = BiRNN_encoder(300, 600, len(vocab), 2, 'LSTM', True, 0.3, 0).to(device)
decoder = BiRNN_decoder(300, 300, len(vocab), 10, 0.3).to(device)

c:\Users\setul\miniconda3\envs\talk-berty\lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [149]:
import torch
import torch.nn as nn
import torch.optim as optim
import random

GRAD_CLIP = 1.
NUM_EPOCHS = 3
TEACHER_FORCING_RATIO = 0.5
MAX_LENGTH = 12

# Assuming the existence of 'encoder' and 'decoder' instances,
# and 'sent_dataloader' as your DataLoader instance for training data.

loss_fn = nn.CrossEntropyLoss(ignore_index=vocab['<pad>'])  # Ignore padding token for loss calculation
learning_rate = 0.001
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
val_losses = []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

for epoch in range(NUM_EPOCHS):
    encoder.train()
    decoder.train()
    total_loss = 0

    for i, batch in enumerate(batch_dataloader):
        src, trg, context = batch
        src, trg, context = src.to(device), trg.to(device), context.to(device)
        print("context",context.shape)
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        output, hidden = encoder(context)

        hidden = repackage_hidden(hidden)  # Prepare hidden state for decoding
        # hidden = hidden.view(input.size(0),3 , -1)

        input = torch.tensor([vocab['<BOS>']] * src.size(0), device=device)  # Initialize decoder input
        print("decoder_input", input.shape)
        print("decoder_hidden_pre", hidden.shape)
        
        loss = 0
        for t in range(1, trg.size(1)):  # Iterate through the target sequence
            output, hidden, _ = decoder(input, hidden)
            loss += loss_fn(output.squeeze(1), trg[:, t])
            teacher_force = random.random() < TEACHER_FORCING_RATIO
            top1 = output.argmax(2).squeeze(1)
            input = trg[:, t] if teacher_force else top1

        loss /= trg.size(1)  # Normalize loss by sequence length
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(encoder.parameters(), GRAD_CLIP)
        torch.nn.utils.clip_grad_norm_(decoder.parameters(), GRAD_CLIP)

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(batch_dataloader)
    print(f'Epoch {epoch+1}, Loss: {average_loss:.4f}')
    val_losses.append(average_loss)


context torch.Size([8, 60])
decoder_input torch.Size([8])
decoder_hidden_pre torch.Size([8, 2400])
decoder_hidden_post torch.Size([1, 8, 2400])
decoder_input_post torch.Size([8, 1])


IndexError: index 1 is out of bounds for dimension 0 with size 1